## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 
from sklearn.svm import LinearSVC

/home/lab_services_student/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-19 13:13:22.523315: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 13:13:22.524776: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 13:13:22.530237: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 13:13:22.547102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting

## Data Transforming

In [2]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]

In [3]:
data

,url,status,url_length,class,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,...,num_digits_in_domain,subdomain,num_subdomains,pcount,top_domain,ptld,spl,is_HTTPS,depth,subdomain_digits
0,0000111servicehelpdesk.godaddysites.com,0,39,phishing,0,7,2,0,0,0,...,0,0000111servicehelpdesk,1,2,com,1,3,0,1,7
1,000011accesswebform.godaddysites.com,0,36,phishing,0,6,2,0,0,0,...,0,000011accesswebform,1,2,com,1,3,0,1,6
2,00003.online,0,12,phishing,0,5,1,0,0,0,...,5,0,0,1,online,1,2,0,1,0
3,0009servicedeskowa.godaddysites.com,0,35,phishing,0,4,2,0,0,0,...,0,0009servicedeskowa,1,2,com,1,3,0,1,4
4,000n38p.wcomhost.com,0,20,phishing,0,5,2,0,0,0,...,0,000n38p,1,2,com,1,3,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,zzufg.com,0,9,phishing,0,0,1,0,0,0,...,0,0,0,1,com,1,2,0,1,0
798459,zzu.li,0,6,phishing,0,0,1,0,0,0,...,0,0,0,1,li,1,2,0,1,0
798460,zzz.co.uk,0,9,phishing,0,0,2,0,0,0,...,0,0,0,2,co.uk,2,3,0,1,0
798461,zzzoolight.co.za,0,16,phishing,0,0,2,0,0,0,...,0,0,0,2,co.za,2,3,0,1,0


In [4]:
# def sp(text):
#     res=[]
#     res[:]=text+' '
#     # return res
#     return " ".join(res)
# df.url=df.url.apply(sp)


In [5]:
df.url

0            0000111servicehelpdesk.godaddysites.com
1               000011accesswebform.godaddysites.com
2                                       00003.online
3                0009servicedeskowa.godaddysites.com
4                               000n38p.wcomhost.com
                             ...                    
798458                                     zzufg.com
798459                                        zzu.li
798460                                     zzz.co.uk
798461                              zzzoolight.co.za
798462    zzzoolight.co.za0-i-fdik.000webhostapp.com
Name: url, Length: 798463, dtype: object

In [6]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
data['top_domain_encoded'] = label_encoder.fit_transform(data['top_domain'])
data['domain_encoded'] = label_encoder.fit_transform(data['domain'])
data['subdomain_encoded'] = label_encoder.fit_transform(data['subdomain'])
data=data.drop(columns=['url','domain','class','ptld','spl','subdomain','top_domain','pcount'])


In [7]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [8]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [9]:
c=data.subdomain_encoded.nunique()
data.subdomain_encoded.nunique()

111203

In [10]:
d=a+b+c
d

421218

In [11]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,is_HTTPS,depth,subdomain_digits,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,7,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,0,1,6,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,1,0,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,0,1,4,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,0,1,5,225,291146,29


In [12]:

y=data['status']
X=data.drop('status', axis=1)


In [13]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=54)

In [14]:
X_train.shape

(558924, 16)

In [15]:
X_test.shape

(239539, 16)

## Traditional Models

In [16]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
# trains in around 2 minutes and has 0.95 accross the board

              precision    recall  f1-score   support

           0       0.96      0.94      0.95    111486
           1       0.95      0.96      0.95    128053

    accuracy                           0.95    239539
   macro avg       0.95      0.95      0.95    239539
weighted avg       0.95      0.95      0.95    239539



In [17]:
vectorizer = TfidfVectorizer(analyzer='char')
X=vectorizer.fit_transform(df.url)
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
y1=df['status']
X1=tfidf_df
X_train1, X_test1, y_train1,y_test1 = train_test_split(X,y,test_size=0.2,random_state=504)


In [18]:
# rf = RandomForestClassifier()
# rf.fit(X_train1, y_train1)
# y_pred1 = rf.predict(X_test1)
# print(classification_report(y_test1, y_pred1))
# took 32 minutes to train on unstructured data, was less accurate for the most part 

In [19]:
# clf = svm.SVC(kernel='linear') 

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
# ran for 5 hours and did not finish training

In [20]:
clf = LinearSVC() 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
# ran for 5 minutes and did not finish training

              precision    recall  f1-score   support

           0       0.89      0.20      0.32    111486
           1       0.58      0.98      0.73    128053

    accuracy                           0.62    239539
   macro avg       0.74      0.59      0.53    239539
weighted avg       0.73      0.62      0.54    239539



In [21]:

clf = SGDClassifier() 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
# took  4 minuted but terrible results

              precision    recall  f1-score   support

           0       0.47      1.00      0.63    111486
           1       0.39      0.00      0.00    128053

    accuracy                           0.47    239539
   macro avg       0.43      0.50      0.32    239539
weighted avg       0.43      0.47      0.30    239539



# Neural Networks

In [22]:
tld_num=data['top_domain_encoded'].nunique()
ld_num=data['domain_encoded'].nunique()
sub_num=data['subdomain_encoded'].nunique()



## LSTM

In [23]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)


In [24]:
Lstm  = Sequential()
Lstm.add(Embedding(d,8))
Lstm.add(Dropout(0.4))
Lstm.add(LSTM(128,return_state=False))
Lstm.add(Dense(24,activation='relu'))
Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.03), bias_regularizer=l2(0.03)))

In [25]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [26]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around8 minutes an epeoch when split 

Epoch 1/30


17467/17467 - 424s - 24ms/step - accuracy: 0.9171 - loss: 0.2073 - precision: 0.9131 - recall: 0.9339 - val_accuracy: 0.9508 - val_loss: 0.1269 - val_precision: 0.9368 - val_recall: 0.9736
Epoch 2/30
17467/17467 - 355s - 20ms/step - accuracy: 0.9738 - loss: 0.0773 - precision: 0.9751 - recall: 0.9758 - val_accuracy: 0.9513 - val_loss: 0.1251 - val_precision: 0.9536 - val_recall: 0.9554
Epoch 3/30
17467/17467 - 461s - 26ms/step - accuracy: 0.9849 - loss: 0.0488 - precision: 0.9864 - recall: 0.9854 - val_accuracy: 0.9469 - val_loss: 0.1393 - val_precision: 0.9591 - val_recall: 0.9409
Epoch 4/30
17467/17467 - 524s - 30ms/step - accuracy: 0.9887 - loss: 0.0387 - precision: 0.9899 - recall: 0.9888 - val_accuracy: 0.9493 - val_loss: 0.1425 - val_precision: 0.9584 - val_recall: 0.9462
Epoch 5/30
17467/17467 - 548s - 31ms/step - accuracy: 0.9904 - loss: 0.0335 - precision: 0.9914 - recall: 0.9905 - val_accuracy: 0.9490 - val_loss: 0.1447 - val_precision: 0.9530 - val_recall: 0.9516
Epoch 5: ea

## CNN

In [27]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)

In [28]:
cnn  = Sequential()
cnn.add(Embedding(d,8))
cnn.add(Dropout(0.4))
cnn.add(Conv1D(128, 5, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(24,activation='relu'))
cnn.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.03), bias_regularizer=l2(0.03)))

In [29]:
cnn.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [30]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyC=cnn.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around 6 minutes an epoch when split 

Epoch 1/30


17467/17467 - 352s - 20ms/step - accuracy: 0.9271 - loss: 0.1894 - precision: 0.9198 - recall: 0.9462 - val_accuracy: 0.9539 - val_loss: 0.1221 - val_precision: 0.9454 - val_recall: 0.9698
Epoch 2/30
17467/17467 - 263s - 15ms/step - accuracy: 0.9742 - loss: 0.0796 - precision: 0.9747 - recall: 0.9771 - val_accuracy: 0.9517 - val_loss: 0.1229 - val_precision: 0.9544 - val_recall: 0.9554
Epoch 3/30
17467/17467 - 191s - 11ms/step - accuracy: 0.9866 - loss: 0.0476 - precision: 0.9882 - recall: 0.9867 - val_accuracy: 0.9481 - val_loss: 0.1288 - val_precision: 0.9496 - val_recall: 0.9535
Epoch 4/30
17467/17467 - 214s - 12ms/step - accuracy: 0.9901 - loss: 0.0362 - precision: 0.9914 - recall: 0.9901 - val_accuracy: 0.9505 - val_loss: 0.1312 - val_precision: 0.9519 - val_recall: 0.9558
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.
